In [ ]:
mport pandas as pd
import string
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import nltk
import subprocess
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter

# plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from sklearn.metrics import precision_score, recall_score, f1_score
# metric (AUC, ROC, sensitivity & specificity)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
!pip install snntorch
import snntorch as snn
from snntorch import surrogate
from snntorch import utils
import snntorch.functional as SF
import math
!pip install ncps

import numpy as np
import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import LTC
import pytorch_lightning as pl
import torch.utils.data as data
torch.manual_seed(42)
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download("punkt")

# try:
#     nltk.data.find('wordnet.zip')
# except:
#     nltk.download('wordnet', download_dir='/kaggle/working/')
#     command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
#     subprocess.run(command.split())
#     nltk.data.path.append('/kaggle/working/')

from nltk.corpus import wordnet as wn


In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('wordnet')
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [ ]:
def create_embedding_matrix(word_index, embedding_dict=None, d_model=100):
    """
     this function create the embedding matrix save in numpy array
    :param word_index: a dictionary with word: index_value
    :param embedding_dict: a dict with word embedding
    :d_model: the dimension of word pretrained embedding, here I just set to 100, we will define again
    :return a numpy array with embedding vectors for all known words
    """
    embedding_matrix = np.zeros((len(word_index) + 1, d_model))
    ## loop over all the words
    for word, index in word_index.items():
        if word in embedding_dict:
            embedding_matrix[index] = embedding_dict[word]
    return embedding_matrix

In [ ]:
def plot_metrics(df_metrics):
    fig, axs = plt.subplots(3, 2, figsize=(15, 15))  # Creating a grid of subplots
    fig.suptitle('Training and Testing Metrics Over Epochs')

    # Plotting Train and Test Loss
    axs[0, 0].plot(df_metrics['Epoch'], df_metrics['Train Loss'], label='Train Loss')
    axs[0, 0].plot(df_metrics['Epoch'], df_metrics['Test Loss'], label='Test Loss', linestyle='--')
    axs[0, 0].set_title('Loss')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()

    # Plotting Train and Test Accuracy
    axs[0, 1].plot(df_metrics['Epoch'], df_metrics['Train Accuracy'], label='Train Accuracy')
    axs[0, 1].plot(df_metrics['Epoch'], df_metrics['Test Accuracy'], label='Test Accuracy', linestyle='--')
    axs[0, 1].set_title('Accuracy')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('Accuracy')
    axs[0, 1].legend()

    # Plotting Train and Test Precision
    axs[1, 0].plot(df_metrics['Epoch'], df_metrics['Train Precision'], label='Train Precision')
    axs[1, 0].plot(df_metrics['Epoch'], df_metrics['Test Precision'], label='Test Precision', linestyle='--')
    axs[1, 0].set_title('Precision')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('Precision')
    axs[1, 0].legend()

    # Plotting Train and Test Recall
    axs[1, 1].plot(df_metrics['Epoch'], df_metrics['Train Recall'], label='Train Recall')
    axs[1, 1].plot(df_metrics['Epoch'], df_metrics['Test Recall'], label='Test Recall', linestyle='--')
    axs[1, 1].set_title('Recall')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('Recall')
    axs[1, 1].legend()

    # Plotting Train and Test F1 Score
    axs[2, 0].plot(df_metrics['Epoch'], df_metrics['Train F1 Score'], label='Train F1 Score')
    axs[2, 0].plot(df_metrics['Epoch'], df_metrics['Test F1 Score'], label='Test F1 Score', linestyle='--')
    axs[2, 0].set_title('F1 Score')
    axs[2, 0].set_xlabel('Epoch')
    axs[2, 0].set_ylabel('F1 Score')
    axs[2, 0].legend()

    # Plotting Epoch Time
    axs[2, 1].plot(df_metrics['Epoch'], df_metrics['Epoch Time'], label='Epoch Time')
    axs[2, 1].set_title('Time per Epoch')
    axs[2, 1].set_xlabel('Epoch')
    axs[2, 1].set_ylabel('Time (sec)')
    axs[2, 1].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the layout
    plt.show()

In [ ]:
def preprocess_text(text):
    # Initial preprocessing to remove special characters and tokenize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if re.match(r'\b\w+\b', token) and token.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

def tokenize_data(train_texts, test_texts, max_len=25):
    # Preprocess texts
    processed_train_texts = [preprocess_text(text) for text in train_texts]
    processed_test_texts = [preprocess_text(text) for text in test_texts]

    # Create and fit tokenizer on the training texts
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(processed_train_texts)
    
    # Convert texts to sequences and pad them
    train_sequences = tokenizer.texts_to_sequences(processed_train_texts)
    test_sequences = tokenizer.texts_to_sequences(processed_test_texts)
    padded_train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len)
    padded_test_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len)
    
    return padded_train_sequences, padded_test_sequences, tokenizer

dfTest = pd.read_csv("/kaggle/input/info-retreval/test_pos_neg.csv", encoding = 'ISO-8859-1').rename(columns={"Sentiment":"Label"})
dfTrain = pd.read_csv("/kaggle/input/info-retreval/train_pos_neg.csv", encoding = 'ISO-8859-1').rename(columns={"Sentiment":"Label"})
dfTrain["Label"]=dfTrain["Label"].map({"POSITIVE":1, "NEGATIVE":0}).sample(frac=1).reset_index(drop=True)
dfTest["Label"]=dfTest["Label"].map({"POSITIVE":1, "NEGATIVE":0}).sample(frac=1).reset_index(drop=True)


glove = pd.read_csv('../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}

MAX_LEN = 25
train_texts = dfTrain.Headline.values.tolist() 
test_texts = dfTest.Headline.values.tolist()

X_train, X_test, tokenizer = tokenize_data(train_texts, test_texts)
embedding_matrix = create_embedding_matrix(tokenizer.word_index, embedding_dict=glove_embedding, d_model=100)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
batch_size = 128

X_train_tensor = torch.tensor(X_train, dtype=torch.long)
X_train_labels_tensor = torch.tensor(dfTrain["Label"], dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, X_train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=3)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
X_test_labels_tensor = torch.tensor(dfTest["Label"], dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, X_test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, num_workers=3)
embedding_matrix_tensor = torch.tensor(embedding_matrix, dtype=torch.float).to(device)

In [ ]:
def prepare_target(target, seq_lengths, washout, batch_first=False):
    """ Preprocess target for offline training.

    Args:
        target (seq_len, batch, output_size): tensor containing
            the features of the target sequence.
        seq_lengths: list of lengths of each sequence in the batch.
        washout: number of initial timesteps during which output of the
            reservoir is not forwarded to the readout. One value per sample.
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False``

    Returns:
        tensor containing the features of the batch's sequences rolled out along
        one axis, minus the washouts and the padded values.
    """

    if batch_first:
        target = target.transpose(0, 1)
    n_sequences = target.size(1)
    target_dim = target.size(2)
    train_len = sum(torch.tensor(seq_lengths) - torch.tensor(washout)).item()

    new_target = torch.zeros(train_len, target_dim, device=target.device)

    idx = 0
    for s in range(n_sequences):
        batch_len = seq_lengths[s] - washout[s]
        new_target[idx:idx + batch_len, :] = target[washout[s]:seq_lengths[s], s, :]
        idx += batch_len

    return new_target


def washout_tensor(tensor, washout, seq_lengths, bidirectional=False, batch_first=False):
    tensor = tensor.transpose(0, 1) if batch_first else tensor.clone()
    if type(seq_lengths) == list:
        seq_lengths = seq_lengths.copy()
    if type(seq_lengths) == torch.Tensor:
        seq_lengths = seq_lengths.clone()
        
    
    for b in range(tensor.size(1)):
        if washout[b] > 0:
            tmp = tensor[washout[b]:seq_lengths[b], b].clone()
            tensor[:seq_lengths[b] - washout[b], b] = tmp
            tensor[seq_lengths[b] - washout[b]:, b] = 0
            seq_lengths[b] -= washout[b]

            if bidirectional:
                tensor[seq_lengths[b] - washout[b]:, b] = 0
                seq_lengths[b] -= washout[b]

    if type(seq_lengths) == list:
        max_len = max(seq_lengths)
    else:
        max_len = max(seq_lengths).item()

    return tensor[:max_len], seq_lengths


import torch
import torch.nn as nn
from torch.nn.utils.rnn import PackedSequence, pad_packed_sequence



class ESN(nn.Module):
    """ Applies an Echo State Network to an input sequence. Multi-layer Echo
    State Network is based on paper
    Deep Echo State Network (DeepESN): A Brief Survey - Gallicchio, Micheli 2017

    Args:
        input_size: The number of expected features in the input x.
        hidden_size: The number of features in the hidden state h.
        output_size: The number of expected features in the output y.
        num_layers: Number of recurrent layers. Default: 1
        nonlinearity: The non-linearity to use ['tanh'|'relu'|'id'].
            Default: 'tanh'
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False``
        leaking_rate: Leaking rate of reservoir's neurons. Default: 1
        spectral_radius: Desired spectral radius of recurrent weight matrix.
            Default: 0.9
        w_ih_scale: Scale factor for first layer's input weights (w_ih_l0). It
            can be a number or a tensor of size '1 + input_size' and first element
            is the bias' scale factor. Default: 1
        lambda_reg: Ridge regression's shrinkage parameter. Default: 1
        density: Recurrent weight matrix's density. Default: 1
        w_io: If 'True', then the network uses trainable input-to-output
            connections. Default: ``False``
        readout_training: Readout's traning algorithm ['gd'|'svd'|'cholesky'|'inv'].
            If 'gd', gradients are accumulated during backward
            pass. If 'svd', 'cholesky' or 'inv', the network will learn readout's
            parameters during the forward pass using ridge regression. The
            coefficients are computed using SVD, Cholesky decomposition or
            standard ridge regression formula. 'gd', 'cholesky' and 'inv'
            permit the usage of mini-batches to train the readout.
            If 'inv' and matrix is singular, pseudoinverse is used.
        output_steps: defines how the reservoir's output will be used by ridge
            regression method ['all', 'mean', 'last'].
            If 'all', the entire reservoir output matrix will be used.
            If 'mean', the mean of reservoir output matrix along the timesteps
            dimension will be used.
            If 'last', only the last timestep of the reservoir output matrix
            will be used.
            'mean' and 'last' are useful for classification tasks.

    Inputs: input, washout, h_0, target
        input (seq_len, batch, input_size): tensor containing the features of
            the input sequence. The input can also be a packed variable length
            sequence. See `torch.nn.utils.rnn.pack_padded_sequence`
        washout (batch): number of initial timesteps during which output of the
            reservoir is not forwarded to the readout. One value per batch's
            sample.
        h_0 (num_layers, batch, hidden_size): tensor containing
             the initial reservoir's hidden state for each element in the batch.
             Defaults to zero if not provided.

        target (seq_len*batch - washout*batch, output_size): tensor containing
            the features of the batch's target sequences rolled out along one
            axis, minus the washouts and the padded values. It is only needed
            for readout's training in offline mode. Use `prepare_target` to
            compute it.

    Outputs: output, h_n
        - output (seq_len, batch, hidden_size): tensor containing the output
        features (h_k) from the readout, for each k.
        - **h_n** (num_layers * num_directions, batch, hidden_size): tensor
          containing the reservoir's hidden state for k=seq_len.
    """

    def __init__(self, input_size, hidden_size, output_size, num_layers=1,
                 nonlinearity='tanh', batch_first=False, leaking_rate=1,
                 spectral_radius=0.9, w_ih_scale=1, lambda_reg=0, density=1,
                 w_io=False, readout_training='svd', output_steps='all'):
        super(ESN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        if nonlinearity == 'tanh':
            mode = 'RES_TANH'
        elif nonlinearity == 'relu':
            mode = 'RES_RELU'
        elif nonlinearity == 'id':
            mode = 'RES_ID'
        else:
            raise ValueError("Unknown nonlinearity '{}'".format(nonlinearity))
        self.batch_first = batch_first
        self.leaking_rate = leaking_rate
        self.spectral_radius = spectral_radius
        if type(w_ih_scale) != torch.Tensor:
            self.w_ih_scale = torch.ones(input_size + 1)
            self.w_ih_scale *= w_ih_scale
        else:
            self.w_ih_scale = w_ih_scale

        self.lambda_reg = lambda_reg
        self.density = density
        self.w_io = w_io
        if readout_training in {'gd', 'svd', 'cholesky', 'inv'}:
            self.readout_training = readout_training
        else:
            raise ValueError("Unknown readout training algorithm '{}'".format(
                readout_training))

        self.reservoir = Reservoir(mode, input_size, hidden_size, num_layers,
                                   leaking_rate, spectral_radius,
                                   self.w_ih_scale, density,
                                   batch_first=batch_first)

        if w_io:
            self.readout = nn.Linear(input_size + hidden_size * num_layers,
                                     output_size)
        else:
            self.readout = nn.Linear(hidden_size * num_layers, output_size)
        if readout_training == 'offline':
            self.readout.weight.requires_grad = False

        if output_steps in {'all', 'mean', 'last'}:
            self.output_steps = output_steps
        else:
            raise ValueError("Unknown task '{}'".format(
                output_steps))

        self.XTX = None
        self.XTy = None
        self.X = None

    def forward(self, input, washout, h_0=None, target=None):
        with torch.no_grad():
            is_packed = isinstance(input, PackedSequence)

            output, hidden = self.reservoir(input, h_0)
            if is_packed:
                output, seq_lengths = pad_packed_sequence(output,
                                                          batch_first=self.batch_first)
            else:
                if self.batch_first:
                    seq_lengths = output.size(0) * [output.size(1)]
                else:
                    seq_lengths = output.size(1) * [output.size(0)]

            if self.batch_first:
                output = output.transpose(0, 1)
                

            output, seq_lengths = washout_tensor(output, washout, seq_lengths)

            if self.w_io:
                if is_packed:
                    input, input_lengths = pad_packed_sequence(input,
                                                          batch_first=self.batch_first)
                else:
                    input_lengths = [input.size(0)] * input.size(1)

                if self.batch_first:
                    input = input.transpose(0, 1)

                input, _ = washout_tensor(input, washout, input_lengths)

                output = torch.cat([input, output], -1)

            if self.readout_training == 'gd' or target is None:
                with torch.enable_grad():
                    output = self.readout(output)

                    if is_packed:
                        for i in range(output.size(1)):
                            if seq_lengths[i] < output.size(0):
                                output[seq_lengths[i]:, i] = 0

                    if self.batch_first:
                        output = output.transpose(0, 1)

                    # Uncomment if you want packed output.
                    # if is_packed:
                    #     output = pack_padded_sequence(output, seq_lengths,
                    #                                   batch_first=self.batch_first)

                    return output, hidden

            else:
                batch_size = output.size(1)

                X = torch.ones(target.size(0), 1 + output.size(2), device=target.device)
                row = 0
                for s in range(batch_size):
                    if self.output_steps == 'all':
                        X[row:row + seq_lengths[s], 1:] = output[:seq_lengths[s],
                                                          s]
                        row += seq_lengths[s]
                    elif self.output_steps == 'mean':
                        X[row, 1:] = torch.mean(output[:seq_lengths[s], s], 0)
                        row += 1
                    elif self.output_steps == 'last':
                        X[row, 1:] = output[seq_lengths[s] - 1, s]
                        row += 1

                if self.readout_training == 'cholesky':
                    if self.XTX is None:
                        self.XTX = torch.mm(X.t(), X)
                        self.XTy = torch.mm(X.t(), target)
                    else:
                        self.XTX += torch.mm(X.t(), X)
                        self.XTy += torch.mm(X.t(), target)

                elif self.readout_training == 'svd':
                    # Scikit-Learn SVD solver for ridge regression.
                    U, s, V = torch.svd(X)
                    idx = s > 1e-15  # same default value as scipy.linalg.pinv
                    s_nnz = s[idx][:, None]
                    UTy = torch.mm(U.t(), target)
                    d = torch.zeros(s.size(0), 1, device=X.device)
                    d[idx] = s_nnz / (s_nnz ** 2 + self.lambda_reg)
                    d_UT_y = d * UTy
                    W = torch.mm(V, d_UT_y).t()

                    self.readout.bias = nn.Parameter(W[:, 0])
                    self.readout.weight = nn.Parameter(W[:, 1:])
                elif self.readout_training == 'inv':
                    self.X = X
                    if self.XTX is None:
                        self.XTX = torch.mm(X.t(), X)
                        self.XTy = torch.mm(X.t(), target)
                    else:
                        self.XTX += torch.mm(X.t(), X)
                        self.XTy += torch.mm(X.t(), target)

                return None, None

    def fit(self):
        if self.readout_training in {'gd', 'svd'}:
            return

        if self.readout_training == 'cholesky':
            W = torch.linalg.solve(self.XTy,
                                   self.XTX + self.lambda_reg * torch.eye(
                                       self.XTX.size(0), device=self.XTX.device))[0].t()
            self.XTX = None
            self.XTy = None

            self.readout.bias = nn.Parameter(W[:, 0])
            self.readout.weight = nn.Parameter(W[:, 1:])
        elif self.readout_training == 'inv':
            I = (self.lambda_reg * torch.eye(self.XTX.size(0))).to(
                self.XTX.device)
            A = self.XTX + I
            X_rank = torch.linalg.matrix_rank(A).item()

            if X_rank == self.X.size(0):
                W = torch.mm(torch.inverse(A), self.XTy).t()
            else:
                W = torch.mm(torch.pinverse(A), self.XTy).t()

            self.readout.bias = nn.Parameter(W[:, 0])
            self.readout.weight = nn.Parameter(W[:, 1:])

            self.XTX = None
            self.XTy = None

    def reset_parameters(self):
        self.reservoir.reset_parameters()
        self.readout.reset_parameters()

        
        
from torch.nn import functional as F
from torch.nn.utils.rnn import PackedSequence
import torch.sparse
import re


def apply_permutation(tensor, permutation, dim=1):
    # type: (Tensor, Tensor, int) -> Tensor
    return tensor.index_select(dim, permutation)


class Reservoir(nn.Module):

    def __init__(self, mode, input_size, hidden_size, num_layers, leaking_rate,
                 spectral_radius, w_ih_scale,
                 density, bias=True, batch_first=False):
        super(Reservoir, self).__init__()
        self.mode = mode
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.leaking_rate = leaking_rate
        self.spectral_radius = spectral_radius
        self.w_ih_scale = w_ih_scale
        self.density = density
        self.bias = bias
        self.batch_first = batch_first

        self._all_weights = []
        for layer in range(num_layers):
            layer_input_size = input_size if layer == 0 else hidden_size

            w_ih = nn.Parameter(torch.Tensor(hidden_size, layer_input_size))
            w_hh = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
            b_ih = nn.Parameter(torch.Tensor(hidden_size))
            layer_params = (w_ih, w_hh, b_ih)

            param_names = ['weight_ih_l{}{}', 'weight_hh_l{}{}']
            if bias:
                param_names += ['bias_ih_l{}{}']
            param_names = [x.format(layer, '') for x in param_names]

            for name, param in zip(param_names, layer_params):
                setattr(self, name, param)
            self._all_weights.append(param_names)

        self.reset_parameters()

    def _apply(self, fn):
        ret = super(Reservoir, self)._apply(fn)
        return ret

    def reset_parameters(self):
        weight_dict = self.state_dict()
        for key, value in weight_dict.items():
            if key == 'weight_ih_l0':
                nn.init.uniform_(value, -1, 1)
                value *= self.w_ih_scale[1:]
            elif re.fullmatch('weight_ih_l[^0]*', key):
                nn.init.uniform_(value, -1, 1)
            elif re.fullmatch('bias_ih_l[0-9]*', key):
                nn.init.uniform_(value, -1, 1)
                value *= self.w_ih_scale[0]
            elif re.fullmatch('weight_hh_l[0-9]*', key):
                w_hh = torch.Tensor(self.hidden_size * self.hidden_size)
                w_hh.uniform_(-1, 1)
                if self.density < 1:
                    zero_weights = torch.randperm(
                        int(self.hidden_size * self.hidden_size))
                    zero_weights = zero_weights[
                                   :int(
                                       self.hidden_size * self.hidden_size * (
                                                   1 - self.density))]
                    w_hh[zero_weights] = 0
                w_hh = w_hh.view(self.hidden_size, self.hidden_size)
                abs_eigs = torch.abs(torch.linalg.eigvals(w_hh))
                weight_dict[key] = w_hh * (self.spectral_radius / torch.max(abs_eigs))

        self.load_state_dict(weight_dict)

    def check_input(self, input, batch_sizes):
        # type: (Tensor, Optional[Tensor]) -> None
        expected_input_dim = 2 if batch_sizes is not None else 3
        if input.dim() != expected_input_dim:
            raise RuntimeError(
                'input must have {} dimensions, got {}'.format(
                    expected_input_dim, input.dim()))
        if self.input_size != input.size(-1):
            raise RuntimeError(
                'input.size(-1) must be equal to input_size. Expected {}, got {}'.format(
                    self.input_size, input.size(-1)))

    def get_expected_hidden_size(self, input, batch_sizes):
        # type: (Tensor, Optional[Tensor]) -> Tuple[int, int, int]
        if batch_sizes is not None:
            mini_batch = batch_sizes[0]
            mini_batch = int(mini_batch)
        else:
            mini_batch = input.size(0) if self.batch_first else input.size(1)
        expected_hidden_size = (self.num_layers, mini_batch, self.hidden_size)
        return expected_hidden_size

    def check_hidden_size(self, hx, expected_hidden_size, msg='Expected hidden size {}, got {}'):
        # type: (Tensor, Tuple[int, int, int], str) -> None
        if hx.size() != expected_hidden_size:
            raise RuntimeError(msg.format(expected_hidden_size, tuple(hx.size())))

    def check_forward_args(self, input, hidden, batch_sizes):
        # type: (Tensor, Tensor, Optional[Tensor]) -> None
        self.check_input(input, batch_sizes)
        expected_hidden_size = self.get_expected_hidden_size(input, batch_sizes)

        self.check_hidden_size(hidden, expected_hidden_size)

    def permute_hidden(self, hx, permutation):
        # type: (Tensor, Optional[Tensor]) -> Tensor
        if permutation is None:
            return hx
        return apply_permutation(hx, permutation)

    def forward(self, input, hx=None):
        is_packed = isinstance(input, PackedSequence)
        if is_packed:
            input, batch_sizes, sorted_indices, unsorted_indices = input
            max_batch_size = int(batch_sizes[0])
        else:
            batch_sizes = None
            max_batch_size = input.size(0) if self.batch_first else input.size(1)
            sorted_indices = None
            unsorted_indices = None

        if hx is None:
            hx = input.new_zeros(self.num_layers, max_batch_size,
                                 self.hidden_size, requires_grad=False)
        else:
            # Each batch of the hidden state should match the input sequence that
            # the user believes he/she is passing in.
            hx = self.permute_hidden(hx, sorted_indices)

        flat_weight = None

        self.check_forward_args(input, hx, batch_sizes)
        func = AutogradReservoir(
            self.mode,
            self.input_size,
            self.hidden_size,
            num_layers=self.num_layers,
            batch_first=self.batch_first,
            train=self.training,
            variable_length=is_packed,
            flat_weight=flat_weight,
            leaking_rate=self.leaking_rate
        )
        output, hidden = func(input, self.all_weights, hx, batch_sizes)
        if is_packed:
            output = PackedSequence(output, batch_sizes, sorted_indices, unsorted_indices)
        return output, self.permute_hidden(hidden, unsorted_indices)

    def extra_repr(self):
        s = '({input_size}, {hidden_size}'
        if self.num_layers != 1:
            s += ', num_layers={num_layers}'
        if self.bias is not True:
            s += ', bias={bias}'
        if self.batch_first is not False:
            s += ', batch_first={batch_first}'
        s += ')'
        return s.format(**self.__dict__)

    def __setstate__(self, d):
        super(Reservoir, self).__setstate__(d)
        self.__dict__.setdefault('_data_ptrs', [])
        if 'all_weights' in d:
            self._all_weights = d['all_weights']
        if isinstance(self._all_weights[0][0], str):
            return
        num_layers = self.num_layers
        self._all_weights = []
        for layer in range(num_layers):
            weights = ['weight_ih_l{}{}', 'weight_hh_l{}{}', 'bias_ih_l{}{}']
            weights = [x.format(layer) for x in weights]
            if self.bias:
                self._all_weights += [weights]
            else:
                self._all_weights += [weights[:2]]

    @property
    def all_weights(self):
        return [[getattr(self, weight) for weight in weights] for weights in
                self._all_weights]


def StackedRNN(inners, num_layers, lstm=False, train=True):
    num_directions = len(inners)
    total_layers = num_layers * num_directions

    def forward(input, hidden, weight, batch_sizes):
        assert (len(weight) == total_layers)
        next_hidden = []
        all_layers_output = []

        for i in range(num_layers):
            all_output = []
            for j, inner in enumerate(inners):
                l = i * num_directions + j

                hy, output = inner(input, hidden[l], weight[l], batch_sizes)
                next_hidden.append(hy)
                all_output.append(output)

            input = torch.cat(all_output, input.dim() - 1)
            all_layers_output.append(input)

        all_layers_output = torch.cat(all_layers_output, -1)
        next_hidden = torch.cat(next_hidden, 0).view(
            total_layers, *next_hidden[0].size())

        return next_hidden, all_layers_output

    return forward

def AutogradReservoir(mode, input_size, hidden_size, num_layers=1,
                      batch_first=False, train=True,
                      batch_sizes=None, variable_length=False, flat_weight=None,
                      leaking_rate=1):
    if mode == 'RES_TANH':
        cell = ResTanhCell
    elif mode == 'RES_RELU':
        cell = ResReLUCell
    elif mode == 'RES_ID':
        cell = ResIdCell

    if variable_length:
        layer = (VariableRecurrent(cell, leaking_rate),)
    else:
        layer = (Recurrent(cell, leaking_rate),)

    func = StackedRNN(layer,
                      num_layers,
                      False,
                      train=train)

    def forward(input, weight, hidden, batch_sizes):
        if batch_first and batch_sizes is None:
            input = input.transpose(0, 1)

        nexth, output = func(input, hidden, weight, batch_sizes)

        if batch_first and not variable_length:
            output = output.transpose(0, 1)

        return output, nexth

    return forward


def Recurrent(inner, leaking_rate):
    def forward(input, hidden, weight, batch_sizes):
        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden = inner(input[i], hidden, leaking_rate, *weight)
            # hack to handle LSTM
            output.append(hidden[0] if isinstance(hidden, tuple) else hidden)

        output = torch.cat(output, 0).view(input.size(0), *output[0].size())

        return hidden, output

    return forward


def VariableRecurrent(inner, leaking_rate):
    def forward(input, hidden, weight, batch_sizes):
        output = []
        input_offset = 0
        last_batch_size = batch_sizes[0]
        hiddens = []
        flat_hidden = not isinstance(hidden, tuple)
        if flat_hidden:
            hidden = (hidden,)
        for batch_size in batch_sizes:
            step_input = input[input_offset:input_offset + batch_size]
            input_offset += batch_size

            dec = last_batch_size - batch_size
            if dec > 0:
                hiddens.append(tuple(h[-dec:] for h in hidden))
                hidden = tuple(h[:-dec] for h in hidden)
            last_batch_size = batch_size

            if flat_hidden:
                hidden = (inner(step_input, hidden[0], leaking_rate, *weight),)
            else:
                hidden = inner(step_input, hidden, leaking_rate, *weight)

            output.append(hidden[0])
        hiddens.append(hidden)
        hiddens.reverse()

        hidden = tuple(torch.cat(h, 0) for h in zip(*hiddens))
        assert hidden[0].size(0) == batch_sizes[0]
        if flat_hidden:
            hidden = hidden[0]
        output = torch.cat(output, 0)

        return hidden, output

    return forward


    num_directions = len(inners)
    total_layers = num_layers * num_directions

    def forward(input, hidden, weight, batch_sizes):
        assert (len(weight) == total_layers)
        next_hidden = []
        all_layers_output = []

        for i in range(num_layers):
            all_output = []
            for j, inner in enumerate(inners):
                l = i * num_directions + j

                hy, output = inner(input, hidden[l], weight[l], batch_sizes)
                next_hidden.append(hy)
                all_output.append(output)

            input = torch.cat(all_output, input.dim() - 1)
            all_layers_output.append(input)

        all_layers_output = torch.cat(all_layers_output, -1)
        next_hidden = torch.cat(next_hidden, 0).view(
            total_layers, *next_hidden[0].size())

        return next_hidden, all_layers_output

    return forward


def ResTanhCell(input, hidden, leaking_rate, w_ih, w_hh, b_ih=None):
    hy_ = torch.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh))
    hy = (1 - leaking_rate) * hidden + leaking_rate * hy_
    return hy


def ResReLUCell(input, hidden, leaking_rate, w_ih, w_hh, b_ih=None):
    hy_ = F.relu(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh))
    hy = (1 - leaking_rate) * hidden + leaking_rate * hy_
    return hy


def ResIdCell(input, hidden, leaking_rate, w_ih, w_hh, b_ih=None):
    hy_ = F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh)
    hy = (1 - leaking_rate) * hidden + leaking_rate * hy_
    return hy

In [ ]:
def train_test_NN_metrics(model, criterion, optimizer, num_epochs, train_loader, test_loader, device, verbose):
    metrics = {
        'Epoch': [],
        'Train Loss': [],
        'Train Accuracy': [],
        'Train Precision': [],
        'Train Recall': [],
        'Train F1 Score': [],
        'Test Loss': [],
        'Test Accuracy': [],
        'Test Precision': [],
        'Test Recall': [],
        'Test F1 Score': [],
        'Epoch Time': []
    }

    for epoch in range(num_epochs):  # num_epochs is the number of epochs you want to train for
        epoch_start_time = time.time()  # Start time of the epoch

        # Training phase
        model.train()
        running_loss = 0
        running_correct = 0
        total_train = 0
        all_train_predicted = []
        all_train_labels = []

        for inputs, labels in train_loader:  # train_loader is your DataLoader for training data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Get model predictions
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update model parameters

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (predicted == labels).sum().item()
            total_train += labels.size(0)
            all_train_predicted.extend(predicted.cpu().numpy())
            all_train_labels.extend(labels.cpu().numpy())
            
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        train_loss = running_loss / total_train
        train_accuracy = running_correct / total_train
        train_precision = precision_score(all_train_labels, all_train_predicted, average='macro')
        train_recall = recall_score(all_train_labels, all_train_predicted, average='macro')
        train_f1 = f1_score(all_train_labels, all_train_predicted, average='macro')
        
        # Testing phase
        model.eval()
        running_loss = 0
        running_correct = 0
        total_test = 0
        all_test_predicted = []
        all_test_labels = []

        with torch.no_grad():  # No need to calculate gradients during testing
            for inputs, labels in test_loader:  # test_loader is your DataLoader for testing data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                running_correct += (predicted == labels).sum().item()
                total_test += labels.size(0)
                all_test_predicted.extend(predicted.cpu().numpy())
                all_test_labels.extend(labels.cpu().numpy())

        test_loss = running_loss / total_test
        test_accuracy = running_correct / total_test
        test_precision = precision_score(all_test_labels, all_test_predicted, average='macro')
        test_recall = recall_score(all_test_labels, all_test_predicted, average='macro')
        test_f1 = f1_score(all_test_labels, all_test_predicted, average='macro')
        

        # Append metrics for this epoch to the dictionary
        metrics['Epoch'].append(epoch + 1)
        metrics['Train Loss'].append(train_loss)
        metrics['Train Accuracy'].append(train_accuracy)
        metrics['Train Precision'].append(train_precision)
        metrics['Train Recall'].append(train_recall)
        metrics['Train F1 Score'].append(train_f1)
        metrics['Test Loss'].append(test_loss)
        metrics['Test Accuracy'].append(test_accuracy)
        metrics['Test Precision'].append(test_precision)
        metrics['Test Recall'].append(test_recall)
        metrics['Test F1 Score'].append(test_f1)
        metrics['Epoch Time'].append(epoch_duration)
        if verbose:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy*100:.2f}%, "
                  f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train F1 Score: {train_f1:.4f}, "
                  f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy*100:.2f}%, Test Precision: {test_precision:.4f}, "
                  f"Test Recall: {test_recall:.4f}, Test F1 Score: {test_f1:.4f}, Epoch Time: {epoch_duration:.2f} sec")

    df_metrics = pd.DataFrame(metrics)
    return df_metrics

In [ ]:
class TextESNClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, num_layers, num_classes, washout, nonlinearity='tanh', leaking_rate=1.0, spectral_radius=0.9, w_ih_scale=1.0, lambda_reg=1.0, density=1.0):
        super(TextESNClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True).to(device)
        self.esn = ESN(input_size=embedding_matrix.size(1), hidden_size=hidden_size, num_layers=num_layers, w_io=True, output_size=hidden_size, output_steps='mean', batch_first=False, nonlinearity=nonlinearity, leaking_rate=leaking_rate, spectral_radius=spectral_radius, w_ih_scale=w_ih_scale, lambda_reg=lambda_reg, density=density)
        self.output_layer = nn.Linear(hidden_size, num_classes)
        self.washout = washout  # Set the washout as a fixed attribute, or make it adjustable as per requirement

    def forward(self, x, h_0=None):
        # Transpose x from [batch_size, seq_len, input_size] to [seq_len, batch_size, input_size]

        # Proceed with embedding if applicable
        x = self.embedding(x)
        x = x.transpose(0, 1)  # Now x is of shape [seq_len, batch_size, input_size]

        # Pass through ESN
        x, _ = self.esn(x, washout=torch.full((x.size(1),), self.washout, dtype=torch.long, device=x.device))  # ESN should be defined with batch_first=False

        # You might need to adjust how you handle outputs depending on your specific model and task
        x = x[-1, :, :]  # Now x is of shape [batch_size, hidden_size]

        # Pass through the output layer
        logits = self.output_layer(x)
        return logits

In [ ]:
def plot_metrics(df_metrics):
    fig, axs = plt.subplots(3, 2, figsize=(15, 15))  # Creating a grid of subplots
    fig.suptitle('Training and Testing Metrics Over Epochs')

    # Plotting Train and Test Loss
    axs[0, 0].plot(df_metrics['Epoch'], df_metrics['Train Loss'], label='Train Loss')
    axs[0, 0].plot(df_metrics['Epoch'], df_metrics['Test Loss'], label='Test Loss', linestyle='--')
    axs[0, 0].set_title('Loss')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()

    # Plotting Train and Test Accuracy
    axs[0, 1].plot(df_metrics['Epoch'], df_metrics['Train Accuracy'], label='Train Accuracy')
    axs[0, 1].plot(df_metrics['Epoch'], df_metrics['Test Accuracy'], label='Test Accuracy', linestyle='--')
    axs[0, 1].set_title('Accuracy')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('Accuracy')
    axs[0, 1].legend()

    # Plotting Train and Test Precision
    axs[1, 0].plot(df_metrics['Epoch'], df_metrics['Train Precision'], label='Train Precision')
    axs[1, 0].plot(df_metrics['Epoch'], df_metrics['Test Precision'], label='Test Precision', linestyle='--')
    axs[1, 0].set_title('Precision')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('Precision')
    axs[1, 0].legend()

    # Plotting Train and Test Recall
    axs[1, 1].plot(df_metrics['Epoch'], df_metrics['Train Recall'], label='Train Recall')
    axs[1, 1].plot(df_metrics['Epoch'], df_metrics['Test Recall'], label='Test Recall', linestyle='--')
    axs[1, 1].set_title('Recall')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('Recall')
    axs[1, 1].legend()

    # Plotting Train and Test F1 Score
    axs[2, 0].plot(df_metrics['Epoch'], df_metrics['Train F1 Score'], label='Train F1 Score')
    axs[2, 0].plot(df_metrics['Epoch'], df_metrics['Test F1 Score'], label='Test F1 Score', linestyle='--')
    axs[2, 0].set_title('F1 Score')
    axs[2, 0].set_xlabel('Epoch')
    axs[2, 0].set_ylabel('F1 Score')
    axs[2, 0].legend()

    # Plotting Epoch Time
    axs[2, 1].plot(df_metrics['Epoch'], df_metrics['Epoch Time'], label='Epoch Time')
    axs[2, 1].set_title('Time per Epoch')
    axs[2, 1].set_xlabel('Epoch')
    axs[2, 1].set_ylabel('Time (sec)')
    axs[2, 1].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the layout
    plt.show()

In [ ]:
model = TextESNClassifier(embedding_matrix_tensor, hidden_size=128,num_layers=2, num_classes=2, washout=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
num_epochs = 100
df_metrics = train_test_NN_metrics(model, criterion, optimizer, num_epochs, train_loader, test_loader, device, verbose=False)
plot_metrics(df_metrics)

In [ ]:
df_metrics.to_csv("TextESNClassifier_pos.csv")


In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        # x is expected to be of shape [seq_len, batch_size, hidden_size]
        query = self.query_layer(x)
        key = self.key_layer(x)
        value = self.value_layer(x)

        # Compute attention scores
        scores = torch.bmm(query.transpose(0, 1), key.transpose(0, 1).transpose(1, 2)) / (query.size(2) ** 0.5)
        attn_weights = F.softmax(scores, dim=-1)

        # Apply attention weights to values
        attended = torch.bmm(attn_weights, value.transpose(0, 1))

        return attended.transpose(0, 1)  # Return to shape [seq_len, batch_size, hidden_size]

class ESNClassifier3(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, num_layers, num_classes, washout):
        super(ESNClassifier3, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True).to(device)
        self.esn1 = ESN(input_size=embedding_matrix.size(1), hidden_size=hidden_size, num_layers=num_layers, w_io=True, output_size=hidden_size, output_steps='mean', batch_first=False)
        self.attention1 = AttentionLayer(hidden_size)
        self.esn2 = ESN(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, w_io=True, output_size=hidden_size, output_steps='mean', batch_first=False)

        # Additional Attention and ESN layers
        self.attention2 = AttentionLayer(hidden_size)
        self.esn3 = ESN(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, w_io=True, output_size=hidden_size, output_steps='mean', batch_first=False)

        self.output_layer = nn.Linear(hidden_size, num_classes)
        self.washout = washout

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)  # [seq_len, batch_size, input_size]

        # First ESN and Attention layer
        esn1_output, _ = self.esn1(x, washout=torch.full((x.size(1),), self.washout, dtype=torch.long, device=x.device))
        attention1_output = self.attention1(esn1_output)
        combined1_output = esn1_output + attention1_output  # Element-wise addition (skip connection)

        # First ESN's output passed to second ESN
        esn2_output, _ = self.esn2(combined1_output, washout=torch.full((x.size(1),), self.washout, dtype=torch.long, device=x.device))

        # Additional Attention and ESN layers
        attention2_output = self.attention2(esn2_output)
        combined2_output = esn2_output + attention2_output  # Element-wise addition (skip connection)
        esn3_output, _ = self.esn3(combined2_output, washout=torch.full((x.size(1),), self.washout, dtype=torch.long, device=x.device))

        # Taking the last output of all sequences for the final classification
        final_output = esn3_output[-1, :, :]

        logits = self.output_layer(final_output)
        return logits


In [ ]:
model = ESNClassifier3(embedding_matrix_tensor, hidden_size=128,num_layers=2, num_classes=2, washout=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
num_epochs = 100
df_metrics = train_test_NN_metrics(model, criterion, optimizer, num_epochs, train_loader, test_loader, device, verbose=False)
plot_metrics(df_metrics)

In [ ]:
df_metrics.to_csv("TextESNClassifier_pos.csv")


In [ ]:
dfTest = pd.read_csv("/kaggle/input/info-retreval/test_neu_opi.csv", encoding = 'ISO-8859-1').rename(columns={"Sentiment":"Label"})
dfTrain = pd.read_csv("/kaggle/input/info-retreval/train_neu_opi.csv", encoding = 'ISO-8859-1').rename(columns={"Sentiment":"Label"})
dfTrain["Label"]=dfTrain["Label"].map({"OPINIONATED":1, "NEUTRAL":0}).sample(frac=1).reset_index(drop=True)
dfTest["Label"]=dfTest["Label"].map({"OPINIONATED":1, "NEUTRAL":0}).sample(frac=1).reset_index(drop=True)


glove = pd.read_csv('../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}

MAX_LEN = 25
train_texts = dfTrain.Headline.values.tolist() 
test_texts = dfTest.Headline.values.tolist()

X_train, X_test, tokenizer = tokenize_data(train_texts, test_texts)
embedding_matrix = create_embedding_matrix(tokenizer.word_index, embedding_dict=glove_embedding, d_model=100)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
batch_size = 128

X_train_tensor = torch.tensor(X_train, dtype=torch.long)
X_train_labels_tensor = torch.tensor(dfTrain["Label"], dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, X_train_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=3)
X_test_tensor = torch.tensor(X_test, dtype=torch.long)
X_test_labels_tensor = torch.tensor(dfTest["Label"], dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, X_test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, num_workers=3)
embedding_matrix_tensor = torch.tensor(embedding_matrix, dtype=torch.float).to(device)

In [ ]:
model = TextESNClassifier(embedding_matrix_tensor, hidden_size=128,num_layers=2, num_classes=2, washout=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
num_epochs = 100
df_metrics = train_test_NN_metrics(model, criterion, optimizer, num_epochs, train_loader, test_loader, device, verbose=False)
plot_metrics(df_metrics)

In [ ]:
df_metrics.to_csv("TextESNClassifier_neu.csv")

In [ ]:
model = ESNClassifier3(embedding_matrix_tensor, hidden_size=128,num_layers=2, num_classes=2, washout=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
num_epochs = 100
df_metrics = train_test_NN_metrics(model, criterion, optimizer, num_epochs, train_loader, test_loader, device, verbose=False)
plot_metrics(df_metrics)

In [ ]:
df_metrics.to_csv("TextESNClassifier3_neu.csv")